In [1]:
import numpy as np 
import pandas as pd

In [4]:
df = pd.read_csv('C:/Users/noura/ml-disaster-survival-project/data/external/Titanic.csv', sep=',')

In [5]:
df.head()

,Passengerid,Age,Fare,Sex,sibsp,zero,zero.1,zero.2,zero.3,zero.4,...,zero.12,zero.13,zero.14,Pclass,zero.15,zero.16,Embarked,zero.17,zero.18,2urvived
0,1,22.0,7.2500,0,1,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,0
1,2,38.0,71.2833,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0.0,0,0,1
2,3,26.0,7.9250,1,0,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,1
3,4,35.0,53.1000,1,1,0,0,0,0,0,...,0,0,0,1,0,0,2.0,0,0,1
4,5,35.0,8.0500,0,0,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,0


In [6]:
import time
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [13]:
print(RAW_DIR.resolve())

C:\Users\noura\ml-disaster-survival-project\data\raw


In [11]:
from pathlib import Path

BASE_DIR = Path("..")  # project root
RAW_DIR = BASE_DIR / "data" / "raw"
RAW_DIR.mkdir(parents=True, exist_ok=True)

lusitania_df.to_csv(RAW_DIR / "lusitania_people_1915.csv", index=False)


In [12]:

BASE = "https://www.rmslusitania.info"

CATEGORY_PATHS = {
    "saloon_1st": "/people/saloon/",
    "second_cabin_2nd": "/people/second-cabin/",
    "third_class_3rd": "/people/third-class/",
    "deck_crew": "/people/deck/",
    "victualling_crew": "/people/victualling/",
    "engineering_crew": "/people/engineering/",
    "band": "/people/band/",
}

def clean_text(x: str) -> str:
    x = re.sub(r"\s+", " ", x.strip())
    return x

def parse_people_table(html: str) -> pd.DataFrame:
    soup = BeautifulSoup(html, "html.parser")

    table = soup.find("table")
    if table is None:
        raise ValueError("No <table> found on page.")

    # Headers
    header_cells = table.find("tr").find_all(["th", "td"])
    headers = [clean_text(h.get_text(" ", strip=True)) for h in header_cells]

    rows = []
    for tr in table.find_all("tr")[1:]:
        tds = tr.find_all("td")
        if not tds:
            continue

        # Survived signal: italics in the NAME cell (first column)
        name_cell = tds[0]
        survived = 1 if (name_cell.find("i") or name_cell.find("em")) else 0

        values = [clean_text(td.get_text(" ", strip=True)) for td in tds]

        # Make row dict robust to header/value length mismatch
        row = {}
        for i, val in enumerate(values):
            key = headers[i] if i < len(headers) else f"col_{i}"
            row[key] = val

        row["survived"] = survived
        rows.append(row)

    return pd.DataFrame(rows)

def fetch_category(name: str, path: str, sleep_s: float = 1.0) -> pd.DataFrame:
    url = BASE + path
    resp = requests.get(url, timeout=30, headers={"User-Agent": "Mozilla/5.0 (educational project)"})
    resp.raise_for_status()
    df = parse_people_table(resp.text)
    df["disaster"] = "lusitania"
    df["category"] = name
    df["source"] = "rmslusitania.info"
    df["year"] = 1915
    time.sleep(sleep_s)  # be polite
    return df

all_dfs = []
for cat_name, path in CATEGORY_PATHS.items():
    print("Fetching:", cat_name, path)
    df_cat = fetch_category(cat_name, path, sleep_s=1.0)
    all_dfs.append(df_cat)

lusitania_df = pd.concat(all_dfs, ignore_index=True)

# Save outputs
lusitania_df.to_csv("data/raw/lusitania_people_1915.csv", index=False)
print("Saved:", lusitania_df.shape, "rows")
lusitania_df.head()


Fetching: saloon_1st /people/saloon/
Fetching: second_cabin_2nd /people/second-cabin/
Fetching: third_class_3rd /people/third-class/
Fetching: deck_crew /people/deck/
Fetching: victualling_crew /people/victualling/
Fetching: engineering_crew /people/engineering/
Fetching: band /people/band/


OSError: Cannot save file into a non-existent directory: 'data\raw'

In [14]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

URL = "https://www.rmslusitania.info/people/saloon/"

# 1) Fetch page
resp = requests.get(
    URL,
    headers={"User-Agent": "Mozilla/5.0 (debugging educational project)"},
    timeout=30
)
resp.raise_for_status()

html = resp.text
print("HTML length:", len(html))

# 2) Parse HTML
soup = BeautifulSoup(html, "html.parser")

# 3) Find the first table
table = soup.find("table")
print("Table found:", table is not None)

# 4) Extract headers
header_row = table.find("tr")
headers = [th.get_text(strip=True) for th in header_row.find_all(["th", "td"])]
print("Headers:", headers)

rows = []

# 5) Iterate over table rows
for tr in table.find_all("tr")[1:]:
    tds = tr.find_all("td")
    if not tds:
        continue

    # Name cell = first column
    name_cell = tds[0]

    # Survival rule: italics indicate survivor
    survived = 1 if (name_cell.find("i") or name_cell.find("em")) else 0

    values = [re.sub(r"\s+", " ", td.get_text(strip=True)) for td in tds]

    row = dict(zip(headers, values))
    row["survived"] = survived
    rows.append(row)

print("Rows parsed:", len(rows))

# 6) Build DataFrame
df_saloon = pd.DataFrame(rows)

df_saloon.head()


HTML length: 223160
Table found: True
Headers: ['Name', 'Ticket number', 'Cabin number', 'Age', 'Citizenship', 'Residence', 'Lifeboat(s)', 'Rescue vessel(s)', 'Body']
Rows parsed: 290


,Name,Ticket number,Cabin number,Age,Citizenship,Residence,Lifeboat(s),Rescue vessel(s),Body,survived
0,"ADAMS, Mr. Arthur Henry",46102,D 37,46,USA,"London, England and Paris, France",17,,,0
1,"ADAMS, Mr. William McMillan(son of Arthur Adams)",46102,D 45,19,USA,"Cambridge, England",17,,,1
2,"ADAMS, Mr. Henry",1298,B 27,58,British,"Tenby, Wales",,,237,0
3,"ADAMS, Mrs. Henry (Annie Elizabeth Macnutt)",1298,B 27,46,British,Canada and USA,,,,1
4,"ALLAN, Lady Hugh Montagu (Marguerite Ethel Mac...",12933,"B 47, B 49 (regal suite and bath)",42,British (Canadian),"Montreal, P.Q., Canada",collapsible,Westborough (Katrina),,1


In [15]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

URL = "https://www.rmslusitania.info/people/second-cabin/"

# 1) Fetch page
resp = requests.get(
    URL,
    headers={"User-Agent": "Mozilla/5.0 (debugging educational project)"},
    timeout=30
)
resp.raise_for_status()

html = resp.text
print("HTML length:", len(html))

# 2) Parse HTML
soup = BeautifulSoup(html, "html.parser")

# 3) Find the table
table = soup.find("table")
print("Table found:", table is not None)

# 4) Extract headers
header_row = table.find("tr")
headers = [th.get_text(strip=True) for th in header_row.find_all(["th", "td"])]
print("Headers:", headers)

rows = []

# 5) Parse rows
for tr in table.find_all("tr")[1:]:
    tds = tr.find_all("td")
    if not tds:
        continue

    name_cell = tds[0]
    survived = 1 if (name_cell.find("i") or name_cell.find("em")) else 0

    values = [re.sub(r"\s+", " ", td.get_text(strip=True)) for td in tds]
    row = dict(zip(headers, values))
    row["survived"] = survived
    rows.append(row)

print("Rows parsed:", len(rows))

# 6) DataFrame
df_second_cabin = pd.DataFrame(rows)

df_second_cabin.head()


HTML length: 246634
Table found: True
Headers: ['Name', 'Age', 'Citizenship', 'Residence', 'Cabin Number', 'Lifeboat', 'Rescue Vessel', 'Body Number']
Rows parsed: 601


,Name,Age,Citizenship,Residence,Cabin Number,Lifeboat,Rescue Vessel,Body Number,survived
0,"ABAS, Mrs. Philip Robert (Beatrice Sanderson)",26,British/Dutch,"New York, NY, USA",,,,,0
1,"ABAS, Miss Isabel",6,British/Dutch,"New York, NY, USA",,,,,0
2,"ABAS, Miss Beatrice",2,British/Dutch,"New York, NY, USA",,,,,0
3,"ABERCROMBY, Mr. Ralph Frank",21,British,"Cincinnati, OH, USA",,,,23,0
4,"ABRAMOWITZ, Mr. Samuel",36,Russian,"Paris, France",,13,Stormcock,,1


In [16]:
CATEGORIES = {
    "saloon_1st": "https://www.rmslusitania.info/people/saloon/",
    "second_cabin_2nd": "https://www.rmslusitania.info/people/second-cabin/",
}

def parse_people_table(url: str) -> pd.DataFrame:
    resp = requests.get(
        url,
        headers={"User-Agent": "Mozilla/5.0 (educational project)"},
        timeout=30
    )
    resp.raise_for_status()

    soup = BeautifulSoup(resp.text, "html.parser")
    table = soup.find("table")
    if table is None:
        raise ValueError(f"No <table> found at {url}")

    header_row = table.find("tr")
    headers = [th.get_text(strip=True) for th in header_row.find_all(["th", "td"])]

    rows = []
    for tr in table.find_all("tr")[1:]:
        tds = tr.find_all("td")
        if not tds:
            continue

        name_cell = tds[0]
        survived = 1 if (name_cell.find("i") or name_cell.find("em")) else 0
        values = [re.sub(r"\s+", " ", td.get_text(strip=True)) for td in tds]

        row = dict(zip(headers, values))
        row["survived"] = survived
        rows.append(row)

    return pd.DataFrame(rows)

# --- Loop over just these two categories ---
dfs = []
for category, url in CATEGORIES.items():
    print(f"Fetching {category}: {url}")
    df = parse_people_table(url)
    df["category"] = category
    df["disaster"] = "lusitania"
    df["year"] = 1915
    df["source"] = "rmslusitania.info"
    print(f"  -> rows: {len(df)} | survived: {df['survived'].sum()} | died: {(df['survived']==0).sum()}")
    dfs.append(df)
    time.sleep(1)  # polite

lusitania_two = pd.concat(dfs, ignore_index=True)
lusitania_two.head()


Fetching saloon_1st: https://www.rmslusitania.info/people/saloon/
  -> rows: 290 | survived: 113 | died: 177
Fetching second_cabin_2nd: https://www.rmslusitania.info/people/second-cabin/
  -> rows: 601 | survived: 230 | died: 371


,Name,Ticket number,Cabin number,Age,Citizenship,Residence,Lifeboat(s),Rescue vessel(s),Body,survived,category,disaster,year,source,Cabin Number,Lifeboat,Rescue Vessel,Body Number
0,"ADAMS, Mr. Arthur Henry",46102,D 37,46,USA,"London, England and Paris, France",17,,,0,saloon_1st,lusitania,1915,rmslusitania.info,NaN,NaN,NaN,NaN
1,"ADAMS, Mr. William McMillan(son of Arthur Adams)",46102,D 45,19,USA,"Cambridge, England",17,,,1,saloon_1st,lusitania,1915,rmslusitania.info,NaN,NaN,NaN,NaN
2,"ADAMS, Mr. Henry",1298,B 27,58,British,"Tenby, Wales",,,237,0,saloon_1st,lusitania,1915,rmslusitania.info,NaN,NaN,NaN,NaN
3,"ADAMS, Mrs. Henry (Annie Elizabeth Macnutt)",1298,B 27,46,British,Canada and USA,,,,1,saloon_1st,lusitania,1915,rmslusitania.info,NaN,NaN,NaN,NaN
4,"ALLAN, Lady Hugh Montagu (Marguerite Ethel Mac...",12933,"B 47, B 49 (regal suite and bath)",42,British (Canadian),"Montreal, P.Q., Canada",collapsible,Westborough (Katrina),,1,saloon_1st,lusitania,1915,rmslusitania.info,NaN,NaN,NaN,NaN


In [17]:
CATEGORIES = {
    "saloon_1st": "https://www.rmslusitania.info/people/saloon/",
    "second_cabin_2nd": "https://www.rmslusitania.info/people/second-cabin/",
    "third_class_3rd": "https://www.rmslusitania.info/people/third-class/",
    "deck_crew": "https://www.rmslusitania.info/people/deck/",
    "victualling_crew": "https://www.rmslusitania.info/people/victualling/",
    "engineering_crew": "https://www.rmslusitania.info/people/engineering/",
    "band": "https://www.rmslusitania.info/people/band/",
    "stowaways": "https://www.rmslusitania.info/people/stowaways/",
    "cameronia_transfers": "https://www.rmslusitania.info/people/cameronia-transfers/",
}

def parse_people_table(url: str) -> pd.DataFrame:
    resp = requests.get(
        url,
        headers={"User-Agent": "Mozilla/5.0 (educational project)"},
        timeout=30
    )
    resp.raise_for_status()

    soup = BeautifulSoup(resp.text, "html.parser")
    table = soup.find("table")
    if table is None:
        return pd.DataFrame()  # no table on this page

    header_row = table.find("tr")
    headers = [th.get_text(strip=True) for th in header_row.find_all(["th", "td"])]

    rows = []
    for tr in table.find_all("tr")[1:]:
        tds = tr.find_all("td")
        if not tds:
            continue

        name_cell = tds[0]
        survived = 1 if (name_cell.find("i") or name_cell.find("em")) else 0
        values = [re.sub(r"\s+", " ", td.get_text(strip=True)) for td in tds]

        row = dict(zip(headers, values))
        row["survived"] = survived
        rows.append(row)

    return pd.DataFrame(rows)

# Ensure output dirs exist (you are in notebooks/, so go one level up)
BASE_DIR = Path("..")
RAW_DIR = BASE_DIR / "data" / "raw"
RAW_DIR.mkdir(parents=True, exist_ok=True)

dfs = []
skipped = []
errors = []

for category, url in CATEGORIES.items():
    print(f"\nFetching {category}: {url}")
    try:
        df = parse_people_table(url)

        if df.empty:
            print("  -> SKIPPED (no <table> found)")
            skipped.append((category, url))
            continue

        df["category"] = category
        df["disaster"] = "lusitania"
        df["year"] = 1915
        df["source"] = "rmslusitania.info"

        # quick sanity line
        print(f"  -> OK rows={len(df)} survived={int(df['survived'].sum())} died={int((df['survived']==0).sum())}")

        # optional: save per-category CSV
        df.to_csv(RAW_DIR / f"lusitania_{category}_1915.csv", index=False)

        dfs.append(df)
        time.sleep(1)  # be polite
    except Exception as e:
        print("  -> ERROR:", repr(e))
        errors.append((category, url, repr(e)))

# Combine everything that worked
if dfs:
    lusitania_all = pd.concat(dfs, ignore_index=True)
    out_path = RAW_DIR / "lusitania_people_1915_all_tables.csv"
    lusitania_all.to_csv(out_path, index=False)
    print("\nSaved combined CSV to:", out_path.resolve())
    print("Total rows:", len(lusitania_all))
else:
    print("\nNo tables were parsed successfully.")

print("\nSkipped (no table):", skipped)
print("Errors:", errors)

# show a preview
lusitania_all.head() if dfs else None



Fetching saloon_1st: https://www.rmslusitania.info/people/saloon/
  -> OK rows=290 survived=113 died=177

Fetching second_cabin_2nd: https://www.rmslusitania.info/people/second-cabin/
  -> OK rows=601 survived=230 died=371

Fetching third_class_3rd: https://www.rmslusitania.info/people/third-class/
  -> OK rows=371 survived=134 died=237

Fetching deck_crew: https://www.rmslusitania.info/people/deck/
  -> OK rows=78 survived=41 died=37

Fetching victualling_crew: https://www.rmslusitania.info/people/victualling/
  -> OK rows=306 survived=138 died=168

Fetching engineering_crew: https://www.rmslusitania.info/people/engineering/
  -> OK rows=313 survived=110 died=203

Fetching band: https://www.rmslusitania.info/people/band/
  -> OK rows=5 survived=3 died=2

Fetching stowaways: https://www.rmslusitania.info/people/stowaways/
  -> SKIPPED (no <table> found)

Fetching cameronia_transfers: https://www.rmslusitania.info/people/cameronia-transfers/
  -> SKIPPED (no <table> found)

Saved combi

,Name,Ticket number,Cabin number,Age,Citizenship,Residence,Lifeboat(s),Rescue vessel(s),Body,survived,...,year,source,Cabin Number,Lifeboat,Rescue Vessel,Body Number,Rescue vessel,Position,Body No.,Instrument
0,"ADAMS, Mr. Arthur Henry",46102,D 37,46,USA,"London, England and Paris, France",17,,,0,...,1915,rmslusitania.info,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"ADAMS, Mr. William McMillan(son of Arthur Adams)",46102,D 45,19,USA,"Cambridge, England",17,,,1,...,1915,rmslusitania.info,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"ADAMS, Mr. Henry",1298,B 27,58,British,"Tenby, Wales",,,237,0,...,1915,rmslusitania.info,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"ADAMS, Mrs. Henry (Annie Elizabeth Macnutt)",1298,B 27,46,British,Canada and USA,,,,1,...,1915,rmslusitania.info,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"ALLAN, Lady Hugh Montagu (Marguerite Ethel Mac...",12933,"B 47, B 49 (regal suite and bath)",42,British (Canadian),"Montreal, P.Q., Canada",collapsible,Westborough (Katrina),,1,...,1915,rmslusitania.info,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
